In [1]:
from utils import *
from training import training
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from tqdm import tqdm
import torchvision.models as models

In [2]:
resnet = models.resnet50(pretrained=True)

c:\Users\skinz\anaconda3\envs\Deep_learning_course\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\skinz\anaconda3\envs\Deep_learning_course\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [14]:
train_data, test_data = get_data()
train_unlabeled_loader, train_labeled_loader, test_loader = get_loader(train_data, test_data, batch_size=50)
device = get_device()

 <function is_available at 0x00000227F023BE50> and can be used by 1 devices
The device used is NVIDIA GeForce GTX 1050


In [8]:
for name, param in resnet.named_parameters():
    param.requires_grad = False
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:
first_conv = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
last_layer = nn.Sequential(
    nn.Linear(in_features=2048, out_features=512),
    nn.Linear(in_features=512, out_features=10)
)
resnet.conv1 = first_conv
resnet.fc = last_layer

In [11]:
for name, param in resnet.named_parameters():print(name, param.requires_grad)

conv1.weight True
bn1.weight False
bn1.bias False
layer1.0.conv1.weight False
layer1.0.bn1.weight False
layer1.0.bn1.bias False
layer1.0.conv2.weight False
layer1.0.bn2.weight False
layer1.0.bn2.bias False
layer1.0.conv3.weight False
layer1.0.bn3.weight False
layer1.0.bn3.bias False
layer1.0.downsample.0.weight False
layer1.0.downsample.1.weight False
layer1.0.downsample.1.bias False
layer1.1.conv1.weight False
layer1.1.bn1.weight False
layer1.1.bn1.bias False
layer1.1.conv2.weight False
layer1.1.bn2.weight False
layer1.1.bn2.bias False
layer1.1.conv3.weight False
layer1.1.bn3.weight False
layer1.1.bn3.bias False
layer1.2.conv1.weight False
layer1.2.bn1.weight False
layer1.2.bn1.bias False
layer1.2.conv2.weight False
layer1.2.bn2.weight False
layer1.2.bn2.bias False
layer1.2.conv3.weight False
layer1.2.bn3.weight False
layer1.2.bn3.bias False
layer2.0.conv1.weight False
layer2.0.bn1.weight False
layer2.0.bn1.bias False
layer2.0.conv2.weight False
layer2.0.bn2.weight False
layer2.0.bn2.

In [21]:
import torch.optim as optim


output_fn = torch.nn.Softmax(dim=1)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=0.005)

In [22]:
epochs = 25
train_class = training(train_loader=train_labeled_loader, val_loader=test_loader, metric= get_accuracy, device=device)
train_class.train(model=resnet, epochs=epochs, optimizer=optimizer, criterion=criterion, output_fn=output_fn, RGB = False, patience_LR = 2, patience_earlystop = 50)

  4%|▍         | 1/25 [00:10<04:04, 10.20s/it]

| Epoch: 1/25 | Train: Loss 95.4247 Accuracy : 0.6000 | Val: Loss 434.3045 Accuracy : 0.2772



  8%|▊         | 2/25 [00:19<03:46,  9.86s/it]

| Epoch: 2/25 | Train: Loss 112.0460 Accuracy : 0.6700 | Val: Loss 436.5665 Accuracy : 0.2963



 12%|█▏        | 3/25 [00:29<03:35,  9.79s/it]

...

| Epoch: 23/25 | Train: Loss 65.8755 Accuracy : 0.7600 | Val: Loss 506.2001 Accuracy : 0.2556



 96%|█████████▌| 24/25 [04:16<00:11, 11.05s/it]

| Epoch: 24/25 | Train: Loss 66.0125 Accuracy : 0.7600 | Val: Loss 506.3904 Accuracy : 0.2561



100%|██████████| 25/25 [04:26<00:00, 10.66s/it]

| Epoch: 25/25 | Train: Loss 66.1818 Accuracy : 0.7600 | Val: Loss 506.5402 Accuracy : 0.2558



In [ ]:
plt.figure(figsize=(16,8))

plt.subplot(1,2,1)
plt.plot(range(1,epochs + 1), train_class.loss_train, label="train loss" )
plt.plot(range(1, epochs +1), train_class.loss_valid, label="valid loss")
plt.xlabel("epochs")
plt.ylabel("Loss")
plt.title("Loss functions")
plt.legend()

plt.subplot(1,2,2)
plt.plot(range(1,epochs + 1), train_class.acc_train, label="train accuracy" )
plt.plot(range(1, epochs +1), train_class.acc_valid, label="valid accuracy")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.title("Accuracy functions")
plt.legend()

#plt.savefig("Metrics functins Resnet.png")
plt.show()